<a href="https://colab.research.google.com/github/shailesh3335/Project/blob/main/Happy_%26_Sad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import tensorflow as tf
import cv2
import imghdr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


**2.Remove dodgy images**

In [ ]:
import cv2
import imghdr


In [ ]:
data_dir = ("/content/drive/MyDrive/Image classifier")

In [ ]:
image_exts = ['jpeg','jpg','bmp','png']

In [ ]:
for image_class in os.listdir(data_dir):
    print(image_class)

In [ ]:
for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir,image_class)):
        image_path = os.path.join(data_dir,image_class,image)
        try:
          img= cv2.imread(image_path)
          tip= imghdr.what(image_path)
          if tip not in image_exts:
             print('Image not in ext list{}'.format(image_path))
             os.remove(image_path)
        except Exception as e:
            print('Issue with image{}'.format(image_path))

**3.Load Data**

In [ ]:
data = tf.keras.utils.image_dataset_from_directory("/content/drive/MyDrive/Image classifier")

In [ ]:
data_iterator = data.as_numpy_iterator()

In [ ]:
batch = data_iterator.next()

In [ ]:
fig, ax =plt.subplots(ncols=4,figsize=(20,20))
for idx,img in enumerate(batch[0][:4]):
  ax[idx].imshow(img.astype(int))
  ax[idx].title.set_text(batch[1][idx])

**4.Scale Data**

In [ ]:
data = data.map(lambda x,y:(x/255,y))

In [ ]:
data.as_numpy_iterator().next()

**5.Split Data**

In [ ]:
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int (len(data)*.1)

In [ ]:
test_size

In [ ]:
train =data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)


**6.Build Deep Learning Model**

In [ ]:
train

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(16,(3,3),1,activation='relu',input_shape=(256,256,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32,(3,3),1,activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16,(3,3),1,activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(1,activation='sigmoid'))



In [ ]:
model.compile('adam',loss=tf.losses.BinaryCrossentropy(),metrics=['accuracy'])

In [ ]:
model.summary()

**7.Train**

In [ ]:
logdir='logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)


In [ ]:
hist= model.fit(train,epochs=20,validation_data=val,callbacks=[tensorboard_callback])

**8.Plot Performance**

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'],color='teal',label='loss')
plt.plot(hist.history['val_loss'],color='orange',label='val_loss')
fig.suptitle('Loss',fontsize=20)
plt.legend(loc='upper left')
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'],color='teal',label='accuracy')
plt.plot(hist.history['val_accuracy'],color='orange',label='val_accuracy')
fig.suptitle('Accuracy',fontsize=20)
plt.legend(loc='upper left')
plt.show()

**9.Evaluate**

In [ ]:
from tensorflow.keras.metrics import Precision,Recall,BinaryAccuracy

In [ ]:
pre= Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
len(test)

In [ ]:
for batch in test.as_numpy_iterator():
    x,y = batch
    yhat= model.predict(x)
    pre.update_state(y,yhat)
    re.update_state(y,yhat)
    acc.update_state(y,yhat)

In [ ]:
print(f'Precision:{pre.result().numpy()},Recall:{re.result().numpy()},Accuracy:{acc.result().numpy()}')

**10.Test**

In [ ]:
img = cv2.imread('/content/drive/MyDrive/Image classifier/happy man.jpg')
#plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))#colour image tha righ code
plt.imshow(img)
plt.show()

In [ ]:
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
yhat = model.predict(np.expand_dims(resize/255, 0))

In [ ]:
yhat

In [ ]:
if yhat > 0.5:
    print(f'Predicted class is Sad')
else:
    print(f'Predicted class is Happy')

**Save The Model**

In [ ]:
from tensorflow.keras.models import load_model
model.save(os.path.join('/content/drive/MyDrive/Image classifier','imageclassifier.h5'))
new_model = load_model('imageclassifier')
new_model.predict(np.expand_dims(resize/255, 0))